# Building a Machine Learning App in R

## BSGP 7030 Summer 2024

### Loading the necessary libraries

In [5]:
library(caret)
library(kernlab)
library(ggplot2)
library(lattice)
library(randomforest)
library(ellipse)


Attaching package: ‘kernlab’


The following object is masked from ‘package:ggplot2’:

    alpha




ERROR: Error in library(randomforest): there is no package called ‘randomforest’
